# ウェブデータ処理２：WebAPIの活用

画像処理で紹介したような深層学習を使ったアルゴリズムでは、メモリやGPGPUなど高スペックなコンピュータを必要としますが、極めて精度の高い結果を導くことができましたね。   
他にも、マシンのリソースや環境を考えなくてよければ、大規模なデータで高速に計算し、高い精度の結果を導くことのできるアルゴリズムは数多くあります。

アプリケーションを開発する場合は、当然このような高性能のアルゴリズムを使いたいと思いますよね。   
ですが、このようなアルゴリズムは、皆さんのスマートフォンやノートPCでは、スペックが足りなくて動かないかもしれません（例えば深層学習の類はそうなることが多いです）。   
それではどうしたらいいでしょうか？

もしそのアプリの利用を想定しているスマホやノートPCがインターネットにつながっていると想定できるのであれば、
データを高スペックなサーバにインターネット経由で送りつけ、サーバで高度な処理を行った後、
その結果をまたインターネット経由で送り返してもらえばいいです。   
このような機能を実現する仕組みをWeb API (Web Application Programming Interface）と呼びます。   
（もちろん、低スペックマシンでオフラインで動作するようなコンパクトで高性能なアルゴリズムを開発することも重要な課題です！）

Google、Microsoft、Amazon、facebook、IBM、Twitter、Yahoo、docomoなど、Webサービスを提供する大手企業のほとんどはWeb APIを提供しています。   
以下に一例をあげます（リンク先にはWebAPI以外のサービスについても書かれています）。

- [Google Cloud API](https://cloud.google.com/apis/docs/overview?hl=ja)
- [Microsoft Azure API Management](https://azure.microsoft.com/ja-jp/services/api-management/)
- [AWS Amazon API Gateway (WebAPIを構築するサービス）](https://aws.amazon.com/jp/api-gateway/)
- [facebook for developpers](https://developers.facebook.com/)
- [IBM Watson API](https://www.ibm.com/watson/jp-ja/)
- [Twitter](https://developer.twitter.com/)
- [Yahoo! Japan](https://developer.yahoo.co.jp/)
- [docomo](https://dev.smt.docomo.ne.jp/?p=docs.api.index)

第2回で`pandas-datareader`でオンラインで株価を取得しましたが、あれもWeb APIの一種です。   

`pandas-datareader`では無料でサービスを利用できましたが、Web APIは基本的に有料のサービスです。   
各社のAPIを利用する場合は、まずそのサイトでアカウントを作成し、場合によってはクレジットカード番号の登録が求められる場合もあります（その場合でも、利用開始後しばらくは無償であったり、一定期間内の利用が一定以内であれば無料、それ以上は利用した分に応じて課金されるような仕組みをとっている場合が多いです）。    
また、無償で利用可能なサイトでは、悪用されないように、個人情報に加え、何のために利用するのかといった説明を記載させ、数日程度の審査を通じてアカウントが発行される場合もあります。   

今回の教材には、Twitter、IBM Watson、Yahooのプログラムを載せていますが、これらのプログラムを実行するためには、アカウントの登録が必要で、その際に各社に皆さんの個人情報を登録する必要があります。   
本授業の一環として皆さんの個人情報を企業に渡すことをお勧めすることは本意ではないので、   
**`WebDataProcessing1.ipynb`と`WebDataProcessing2.ipynb`以外のファイルは今回は実行不要です。**   

一方、`pandas-datareader`のように、無償・アカウント不要でWeb APIサービスを提供してくれているサイトもあります。   
今回はその一つである[Livedoor天気情報のWether Hacks](http://weather.livedoor.com/weather_hacks/)を利用させていただくことにしましょう。   
くれぐれもwhileループなどで高頻度でアクセスしないでくださいね！


## 1.  Libedoor天気情報の実行

今回は[Livedoor天気情報のWether Hacksの[お天気Webサービス（REST）](http://weather.livedoor.com/weather_hacks/webservice)」を使います。   
以下でこのサイトに記載されている情報を参照します（2019/7アクセス）。   

```
お天気Webサービス（Livedoor Weather Web Service / LWWS）は、現在全国142カ所の今日・明日・あさっての天気予報・予想気温と都道府県の天気概況情報を提供しています。

```
詳しくは[お天気Webサービス仕様](http://weather.livedoor.com/weather_hacks/webservice)をご覧ください。   
値域(city)を指定するパラメータは、[こちら](http://weather.livedoor.com/forecast/rss/primary_area.xml)から見つけることができます。   
```
<pref title="東京都">
<warn title="警報・注意報" source="http://weather.livedoor.com/forecast/rss/warn/13.xml"/>
<city title="東京" id="130010" source="http://weather.livedoor.com/forecast/rss/area/130010.xml"/>
<city title="大島" id="130020" source="http://weather.livedoor.com/forecast/rss/area/130020.xml"/>
<city title="八丈島" id="130030" source="http://weather.livedoor.com/forecast/rss/area/130030.xml"/>
<city title="父島" id="130040" source="http://weather.livedoor.com/forecast/rss/area/130040.xml"/>
</pref>
```
とあるので、東京のIDは「130010」であることがわかります。

「東京都・東京の天気」を取得する場合   
下記URLにアクセスしてJSONデータを取得します。   
```
基本URL + 東京のID（130010）
http://weather.livedoor.com/forecast/webservice/json/v1?city=130010
```

結果はjson形式で返されます。


In [1]:
import requests

# 東京都の
url = 'http://weather.livedoor.com/forecast/webservice/json/v1?city=130010'
api_data = requests.get(url).json()


jsonモジュールを使って、結果を出力してみましょう。  
まずはデータの全貌です。

In [2]:
import json

print(json.dumps(api_data, indent=2, ensure_ascii=False))

{
  "pinpointLocations": [
    {
      "link": "http://weather.livedoor.com/area/forecast/1310100",
      "name": "千代田区"
    },
    {
      "link": "http://weather.livedoor.com/area/forecast/1310200",
      "name": "中央区"
    },
    {
      "link": "http://weather.livedoor.com/area/forecast/1310300",
      "name": "港区"
    },
    {
      "link": "http://weather.livedoor.com/area/forecast/1310400",
      "name": "新宿区"
    },
    {
      "link": "http://weather.livedoor.com/area/forecast/1310500",
      "name": "文京区"
    },
    {
      "link": "http://weather.livedoor.com/area/forecast/1310600",
      "name": "台東区"
    },
    {
      "link": "http://weather.livedoor.com/area/forecast/1310700",
      "name": "墨田区"
    },
    {
      "link": "http://weather.livedoor.com/area/forecast/1310800",
      "name": "江東区"
    },
    {
      "link": "http://weather.livedoor.com/area/forecast/1310900",
      "name": "品川区"
    },
    {
      "link": "http://weather.livedoor.com/area/forecast/1311000",


要素を取り出してみましょう。

In [3]:
import json

for d in api_data['forecasts']:
    if '今日' in d['dateLabel']:
        print('今日の天気：', d['telop'])
    elif '明日' in d['dateLabel']:
        print('明日の天気：', d['telop'])
    elif '明後日' in d['dateLabel']:
        print('明後日の天気：', d['telop'])

print('--------- 天気の概要 -------------')
print(api_data['description']["text"].replace('\n\n','\n'))


今日の天気： 雨
明日の天気： 曇時々雨
明後日の天気： 曇のち雨
--------- 天気の概要 -------------
 梅雨前線が本州の南岸に停滞しています。また、寒気を伴った上空の気圧の谷が対馬海峡付近にあって東へ進んでいます。
 東京地方は曇りや雨となっています。
 16日は、梅雨前線や寒気を伴った上空の気圧の谷が接近するため、雨で夕方まで雷を伴う所があるでしょう。伊豆諸島では、雨や雷雨となる所がある見込みです。
 17日は、梅雨前線や湿った空気の影響で、曇りで未明は雨となるでしょう。伊豆諸島では、昼前まで雨や雷雨となる所があるでしょう。
【関東甲信地方】
 関東甲信地方は曇りで、雨や雷雨となっている所があります。
 16日は、梅雨前線や湿った空気の影響で、雨や曇りとなる見込みです。寒気を伴った上空の気圧の谷が接近するため、雷を伴う所があるでしょう。
 17日は、梅雨前線や湿った空気の影響で、はじめ雨が降りますが、次第に曇りとなる見込みです。伊豆諸島では、雷を伴う所があるでしょう。
 関東地方と伊豆諸島の海上では、17日にかけて波がやや高く、17日はうねりを伴うでしょう。また、所々で霧が発生しています。16日は、船舶は視程障害に注意してください。


## 2. 国立国会図書館サーチ API

[国立国会図書館は](https://iss.ndl.go.jp/information/api/)、「検索用API」「ハーベスト用API」「書影API」の3種類のAPIを提供しています。   
同ページによると、2019年7月現在、「(2) 個人・非営利団体等の方＞データ利用により利益を得ない場合」は利用申請は不要とあります。   
規約は変わる可能性があるのでご注意ください。

### 検索用API

検索条件（検索クエリとも呼びます）はqueryで指定しています。   
[国立国会図書館 API仕様の概要](https://iss.ndl.go.jp/information/api/)を参照し、いろいろと変えてみましょう。


In [4]:
import requests
import urllib.parse

# クエリは題名（title)に「東京大学」を含み、2018年以降の本とします
query = urllib.parse.quote('title="東京大学" AND from="2018"')

# urlにqueryをつないでリクエストを送ります
# 「maximumRecords=10」は、検索結果の書誌情報を10件返してほしいという意味です。
url = 'https://iss.ndl.go.jp/api/sru?operation=searchRetrieve&maximumRecords=10&query='+query
res = requests.get(url)


In [5]:
# 送り返されてきたデータをテキスト形式で表示してみましょう。
# これはXML形式です。
print(res.text)

<?xml version="1.0" encoding="UTF-8"?>
<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
  <version>1.2</version>
  <numberOfRecords>450</numberOfRecords>
  <nextRecordPosition>11</nextRecordPosition>
  <extraResponseData>
&lt;facets&gt;
  &lt;lst name=&quot;REPOSITORY_NO&quot;&gt;
    &lt;int name=&quot;R000000004&quot;&gt;266&lt;/int&gt;
    &lt;int name=&quot;R100000002&quot;&gt;91&lt;/int&gt;
    &lt;int name=&quot;R100000001&quot;&gt;49&lt;/int&gt;
    &lt;int name=&quot;R000000025&quot;&gt;43&lt;/int&gt;
    &lt;int name=&quot;R000000016&quot;&gt;33&lt;/int&gt;
    &lt;int name=&quot;R100000067&quot;&gt;4&lt;/int&gt;
    &lt;int name=&quot;R100000004&quot;&gt;1&lt;/int&gt;
    &lt;int name=&quot;R000000007&quot;&gt;1&lt;/int&gt;
    &lt;int name=&quot;R100000040&quot;&gt;1&lt;/int&gt;
  &lt;/lst&gt;
  &lt;lst name=&quot;NDC&quot;&gt;
    &lt;int name=&quot;0&quot;&gt;2&lt;/int&gt;
    &lt;int name=&quot;2&quot;&gt;15&lt;/int&gt;
    &lt;int name=&quot;3&quot;&gt;36&lt

XML形式をそのままでは見づらいので、XMLパーザを使ってパースします。   
ここでは`lxml`というパーザを使います。   
検索結果として帰ってきた10件の本について、その題名（title)、著者（creator, 情報がない可能性がある）、言語（情報がない可能性がある）を出力してみましょう。

In [6]:
from lxml import etree
 
root = etree.fromstring(res.content)

num = 1
for record in root.findall('.//recordData', root.nsmap):
    print(record.find('recordPosition'))
    print('==== Search Result: '+str(num)+' ================')
    num += 1
    book = etree.fromstring(record.text)
    print("title:" + str(book.find('./{*}title').text)) # 題名
    if book.find(('./{*}creator')) is not None: # 著者名（情報がない可能性がある）
        print("creator:" + str(book.find('./{*}creator').text))
    if book.find(('./{*}language')) is not None: # 著者名（情報がない可能性がある）
        print("language:" + str(book.find('./{*}language').text))


None
==== Search Result: 1 ================
title:2019年 比較腫瘍学常陸宮賞 : 第22回の受賞者は東京大学薬学部教授の三浦正幸博士
language:jpn
None
==== Search Result: 2 ================
title:BOOKS & TRENDS 『志士から英霊へ』を書いた東京大学大学院人文社会系研究科教授 小島毅氏に聞く
creator:小島 毅
language:jpn
None
==== Search Result: 3 ================
title:BOOKS & TRENDS 『トランプのアメリカに住む』を書いた 東京大学大学院情報学環教授 吉見俊哉氏に聞く
creator:吉見 俊哉
language:jpn
None
==== Search Result: 4 ================
title:CLINIC NOTEセミナー紹介 膵炎の診断と治療 : Basics&Topics : 講師:大野耕一先生(東京大学動物医療センター 消化器内科、アジア獣医内科専門医(内科学))
language:jpn
None
==== Search Result: 5 ================
title:Challenger 人の意識を機械に移植 20年後の実現も夢ではない 東京大学大学院工学系研究科 准教授 渡辺正峰氏
creator:渡辺 正峰,今井 拓司
language:jpn
None
==== Search Result: 6 ================
title:Front Runner 挑む(第78回)菅裕明 東京大学大学院理学系研究科教授 創薬の異端「ペプチド」を先端にする
language:jpn
None
==== Search Result: 7 ================
title:Front Runner 挑む(第81回)入江直樹 東京大学大学院理学系研究科 准教授 生物の進化を予測する
language:jpn
None
==== Search Result: 8 ================
title:Front Runner 挑む(第86回)宮本英昭 東京大学教授 原動力は宇宙への憧